In [1]:
import pandas as pd
import torch
from transformers import GPT2LMHeadModel, GPT2Tokenizer
import zipfile
import os

# 💾 Paso 1: Subir el archivo ZIP de Kaggle
zip_path = "/content/spa-eng.zip"  # Cambia esto según el nombre del archivo ZIP subido
dataset_dir = "/content/dataset/"

# Descomprimir el archivo
if os.path.exists(dataset_dir):
    import shutil
    shutil.rmtree(dataset_dir)

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(dataset_dir)

print("Archivos extraídos:", os.listdir(dataset_dir))

Archivos extraídos: ['spa.txt', '_about.txt']


In [3]:
# Cargar el archivo TXT desde ManyThings.org (ejemplo: spa_eng.txt)
dataset_file = "/content/dataset/spa.txt"
df = pd.read_csv(dataset_file, sep='\t', header=None, usecols=[0, 1], names=["english", "spanish"])

# Filtrar datos vacíos y duplicados
df = df.dropna().drop_duplicates()

# Verificar los primeros 5 datos filtrados
print(f"Primeros 5 registros filtrados del dataset: \n{df.head()}")

Primeros 5 registros filtrados del dataset: 
  english  spanish
0     Go.      Ve.
1     Go.    Vete.
2     Go.    Vaya.
3     Go.  Váyase.
4     Hi.    Hola.


In [9]:
# 🏗️ Paso 4: Cargar y afinar el modelo GPT-2
tokenizer = GPT2Tokenizer.from_pretrained("distilgpt2")
model = GPT2LMHeadModel.from_pretrained("distilgpt2")

# Asignar el token de padding como el token de fin de oración (eos_token)
tokenizer.pad_token = tokenizer.eos_token  # Usamos el eos_token como pad_token

In [10]:
# 🏗️ Paso 3: Preprocesar los datos de entrada y salida (traducción)
# Preprocesar los datos de entrada y salida (traducción)
english_train = df["english"].astype(str).tolist()
spanish_train = df["spanish"].astype(str).tolist()

# Usar 1000 ejemplos para entrenamiento (ajusta según tu RAM)
english_train = df["english"].astype(str).tolist()[:1000]
spanish_train = df["spanish"].astype(str).tolist()[:1000]

# Tokenizar con longitud máxima reducida
inputs = tokenizer(english_train, return_tensors="pt", padding=True, truncation=True, max_length=32)
labels = tokenizer(spanish_train, return_tensors="pt", padding=True, truncation=True, max_length=32)

# Verificación de los datos de entrenamiento
print(f"Primeras 5 frases en inglés: {english_train[:5]}")
print(f"Primeras 5 frases en español: {spanish_train[:5]}")


Primeras 5 frases en inglés: ['Go.', 'Go.', 'Go.', 'Go.', 'Hi.']
Primeras 5 frases en español: ['Ve.', 'Vete.', 'Vaya.', 'Váyase.', 'Hola.']


In [11]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer
import torch
from torch.utils.data import DataLoader, TensorDataset

# 🏗️ Paso 4: Cargar y afinar el modelo GPT-2
tokenizer = GPT2Tokenizer.from_pretrained("distilgpt2")
model = GPT2LMHeadModel.from_pretrained("distilgpt2")

# Asignar el token de padding como el token de fin de oración (eos_token)
tokenizer.pad_token = tokenizer.eos_token  # Usamos el eos_token como pad_token

# Tamaño de lote reducido
batch_size = 512  # Ajusta según la capacidad de tu máquina

# Tokenizar los datos de entrenamiento (inglés -> español)
inputs = tokenizer(english_train, return_tensors="pt", padding=True, truncation=True, max_length=64)
labels = tokenizer(spanish_train, return_tensors="pt", padding=True, truncation=True, max_length=64)

# Crear un DataLoader para manejar los lotes
dataset = TensorDataset(inputs['input_ids'], labels['input_ids'])
train_dataloader = DataLoader(dataset, batch_size=batch_size)

# Definir el optimizador
optimizer = torch.optim.AdamW(model.parameters(), lr=5e-5)


In [12]:
# 🎯 Paso 5: Entrenar el modelo
for epoch in range(3):  # Ajusta según necesidad
    model.train()
    optimizer.zero_grad()

    # Entrenamiento del modelo (GPT2 con las secuencias de texto en inglés como entrada y español como salida)
    outputs = model(input_ids=inputs['input_ids'], labels=labels['input_ids'])
    loss = outputs.loss
    loss.backward()
    optimizer.step()
    print(f"Epoch {epoch + 1}, Loss: {loss.item()}")

`loss_type=None` was set in the config but it is unrecognised.Using the default loss: `ForCausalLMLoss`.


ValueError: Expected input batch_size (6000) to match target batch_size (13000).

In [ ]:
# 💾 Paso 6: Guardar el modelo para subirlo a GitHub
model.save_pretrained("/content/gpt2-spanish-translator")
tokenizer.save_pretrained("/content/gpt2-spanish-translator")

print("✅ Modelo entrenado y guardado.")

In [ ]:
# 🚀 Paso 7: Probar el modelo con una nueva frase en inglés
def translate_text(text):
    model.eval()
    input_tokens = tokenizer(text, return_tensors="pt", padding=True, truncation=True, max_length=128)
    output_tokens = model.generate(input_ids=input_tokens['input_ids'], max_length=50)
    return tokenizer.decode(output_tokens[0], skip_special_tokens=True)

# Ejemplo de traducción
print("Traducción generada:", translate_text("Hello, how are you?"))